In [1]:
import numpy as np
import pandas as pd
import pickle 

In [2]:
df = pickle.load(open('dataset_level2.pkl','rb'))
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city,venue
0,2,Australia,Sri Lanka,0.1,0,0,NaN,Melbourne Cricket Ground
1,2,Australia,Sri Lanka,0.2,0,0,NaN,Melbourne Cricket Ground
2,2,Australia,Sri Lanka,0.3,1,0,NaN,Melbourne Cricket Ground
3,2,Australia,Sri Lanka,0.4,2,0,NaN,Melbourne Cricket Ground
4,2,Australia,Sri Lanka,0.5,0,0,NaN,Melbourne Cricket Ground


In [3]:
df[df['city'].isnull()]['venue'].value_counts()

venue
Dubai International Cricket Stadium        3092
Pallekele International Cricket Stadium    2066
Melbourne Cricket Ground                   1453
Sydney Cricket Ground                       749
Adelaide Oval                               498
Harare Sports Club                          372
Sylhet International Cricket Stadium        128
Sharjah Cricket Stadium                     127
Carrara Oval                                 64
Name: count, dtype: int64

In [4]:
x = np.where(df['city'].isnull(),df['venue'].str.split().apply(lambda x:x[0]),df['city'])
df['city'] = x

In [5]:
df.isnull().sum()

match_id            0
batting_team        0
bowling_team        0
ball                0
runs                0
player_dismissed    0
city                0
venue               0
dtype: int64

In [6]:
df.drop(columns=['venue'],inplace=True)
df.head()

,match_id,batting_team,bowling_team,ball,runs,player_dismissed,city
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne


In [7]:
total = df.groupby('match_id').sum()['runs'].reset_index()
df = df.merge(total,on = 'match_id')
df.head()

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,168
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,168
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,168


In [8]:
df['runs_x'] = pd.to_numeric(df['runs_x'], errors='coerce')
df['current_score'] = df.groupby('match_id')['runs_x'].cumsum() 
df['over'] = df['ball'].apply(lambda x:str(x).split(".")[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split('.')[1])
df['balls_bowled'] = (df['over'].astype('int')*6) + df['ball_no'].astype('int')
df['crr'] = round((df['current_score']*6)/df['balls_bowled'],2)
df.head()

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y,current_score,over,ball_no,balls_bowled,crr
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168,0,0,1,1,0.0
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168,0,0,2,2,0.0
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,168,1,0,3,3,2.0
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,168,3,0,4,4,4.5
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,168,3,0,5,5,3.6


In [9]:
df['player_dismissed'] = df['player_dismissed'].astype(str)
df['player_dismissed'] = df['player_dismissed'].apply(lambda x: 0 if x == '0' else 1)
df['player_dismissed'] = df['player_dismissed'].astype(int)
df['player_dismissed'] = df.groupby('match_id')['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']
df.head()

,match_id,batting_team,bowling_team,ball,runs_x,player_dismissed,city,runs_y,current_score,over,ball_no,balls_bowled,crr,wickets_left
0,2,Australia,Sri Lanka,0.1,0,0,Melbourne,168,0,0,1,1,0.0,10
1,2,Australia,Sri Lanka,0.2,0,0,Melbourne,168,0,0,2,2,0.0,10
2,2,Australia,Sri Lanka,0.3,1,0,Melbourne,168,1,0,3,3,2.0,10
3,2,Australia,Sri Lanka,0.4,2,0,Melbourne,168,3,0,4,4,4.5,10
4,2,Australia,Sri Lanka,0.5,0,0,Melbourne,168,3,0,5,5,3.6,10


In [10]:
final_df = df[['match_id','batting_team','bowling_team','runs_x','current_score','balls_bowled','wickets_left','crr','city','runs_y']]

In [11]:
final_df.sample(final_df.shape[0])

,match_id,batting_team,bowling_team,runs_x,current_score,balls_bowled,wickets_left,crr,city,runs_y
12606,142,West Indies,India,0,35,44,7,4.77,Kolkata,109
22799,427,India,England,0,28,39,7,4.31,Ahmedabad,156
15815,214,India,West Indies,1,134,92,6,8.74,Thiruvananthapuram,170
96332,2566,Australia,India,1,56,31,9,10.84,Chandigarh,160
37894,863,India,West Indies,0,40,24,8,10.00,Basseterre,138
...,...,...,...,...,...,...,...,...,...,...
3206,31,New Zealand,Australia,0,101,75,4,8.08,Auckland,150
65707,2211,Sri Lanka,West Indies,0,48,43,10,6.70,London,158
54708,1804,England,India,0,117,83,4,8.46,Chennai,165
8035,88,India,Bangladesh,4,111,88,9,7.57,Colombo,176


In [14]:
final_df['balls_left'] = 120 - final_df['balls_bowled']
final_df['balls_left'] = final_df['balls_left'].apply(lambda x:0 if x<0 else x)
final_df['crr'] = round((final_df['current_score']*6)/final_df['balls_bowled'],2)
final_df.drop(columns=['balls_bowled'],inplace=True)
final_df.head()

C:\Users\vishw\AppData\Local\Temp\ipykernel_16792\1793150933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['balls_left'] = 120 - final_df['balls_bowled']
C:\Users\vishw\AppData\Local\Temp\ipykernel_16792\1793150933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['balls_left'] = final_df['balls_left'].apply(lambda x:0 if x<0 else x)
C:\Users\vishw\AppData\Local\Temp\ipykernel_16792\1793150933.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,balls_left
0,2,Australia,Sri Lanka,0,0,10,0.0,Melbourne,168,119
1,2,Australia,Sri Lanka,0,0,10,0.0,Melbourne,168,118
2,2,Australia,Sri Lanka,1,1,10,2.0,Melbourne,168,117
3,2,Australia,Sri Lanka,2,3,10,4.5,Melbourne,168,116
4,2,Australia,Sri Lanka,0,3,10,3.6,Melbourne,168,115


In [17]:
print(final_df.dtypes)

match_id           int64
batting_team      object
bowling_team      object
runs_x             int64
current_score      int64
wickets_left       int32
crr              float64
city              object
runs_y             int64
balls_left         int64
dtype: object


In [18]:
groups = final_df.groupby('match_id')
match_ids = final_df['match_id'].unique()
last_five = []
for id in match_ids:
    group = groups.get_group(id) 
    group = group[['runs_x']].copy() 
    group['rolling_sum'] = group['runs_x'].rolling(window=30).sum()
    last_five.extend(group['rolling_sum'].values.tolist()) 

final_df['last_five'] = last_five
final_df.head()

C:\Users\vishw\AppData\Local\Temp\ipykernel_16792\1165746440.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['last_five'] = last_five


,match_id,batting_team,bowling_team,runs_x,current_score,wickets_left,crr,city,runs_y,balls_left,last_five
0,2,Australia,Sri Lanka,0,0,10,0.0,Melbourne,168,119,NaN
1,2,Australia,Sri Lanka,0,0,10,0.0,Melbourne,168,118,NaN
2,2,Australia,Sri Lanka,1,1,10,2.0,Melbourne,168,117,NaN
3,2,Australia,Sri Lanka,2,3,10,4.5,Melbourne,168,116,NaN
4,2,Australia,Sri Lanka,0,3,10,3.6,Melbourne,168,115,NaN


In [19]:
final_df.dropna(inplace=True)

C:\Users\vishw\AppData\Local\Temp\ipykernel_16792\2709626079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.dropna(inplace=True)


In [20]:
eligible_cities = final_df['city'].value_counts()[final_df['city'].value_counts()>600].index.tolist()
final_df = final_df[final_df['city'].isin(eligible_cities)]

In [21]:
pickle.dump(final_df,open('dataset_final.pkl','wb'))